Requirements: 

iypnb file - IMS \n
json file  - inventory.json\n
json file  - order.json\n


# Functions:

-> main()
-> userMain()
    -> viewInventory()
    -> addProductToCart(cart)
    -> removeProductFromCart(cart)
    -> viewcart(cart)
    -> payBill(cart)
    
-> adminMain()
    -> addToInventory()
    -> viewOrders()
    -> viewStatistics()
    
-> price(cart)
-> viewCart(cart)
-> load(fileName)
-> saveInOrders(cart)
-> saveInJson(filename,dictionary)

In [ ]:
####################################################################################
import json
from tabulate import tabulate
from matplotlib import pyplot as plt
import os

def main():
    print("-----Welcome To IMS Enterprises----")
    while(True):
        print("-----------------")
        print("Press 1 if User")
        print("Press 2 if Admin")
        print("Press 3 to exit")
        print("-----------------")
        print()
        ch=input()
        if(ch=='1'):
            userMain()
        elif(ch=='2'):
            adminMain()
        elif(ch=='3'):
            print("Come again soon!")
            break
        else:
            print("Invalid choice. Please re-enter")
        print()

def viewInventory():
    with open("inventory.json",'r') as file:
        inventory=json.load(file)
    file.close()
    keys=list(inventory.keys())
    table=[["Id","Name","Quantity","Price"]]
    for i in keys:
        row=[i]
        row.append(inventory[i]['Name'])
        row.append(inventory[i]['Quantity'])
        row.append(inventory[i]['Price'])
        table.append(row)
    print(tabulate(table))

def load(fileName=""):
    file=None
    try:
        with open(fileName,"r") as p:
            file=json.load(p)
        return file
    except:
        return False

def addProductToCart(cart):
    
    viewInventory()
    print("Enter product ID and Quantity")
    pid=input()
    quantity=int(input())
    inventory=load("inventory.json")
    
    if(inventory==False):
        print("Not possible")
    else:
        if(inventory.get(pid,False)):
            if(cart.get(pid,0)+quantity<int(inventory[pid]["Quantity"])):
                cart[pid]=cart.get(pid,0)+quantity
            else:
                print("Insufficient Quatity")
        else:
            print("Invalid ID")

def removeProductFromCart(cart):
    viewCart(cart)
    print("Enter ID and Quantity of item to be removed")
    pid=input()
    qty=int(input())
    if(cart.get(pid,False)):
        if(int(cart[pid])<qty):
            print("Insufficeint Quantity")
        elif(int(cart[pid])==qty):
            del(cart[pid])
        else:
            cart[pid]=int(cart[pid])-qty
    else:
        print("Product not in cart")
             
def saveInOrders(cart):
    order = load("order.json")
    inventory=load("inventory.json")
    if(inventory==False):
        print("Order not valid")
    else:
        if(order==False):
            order={}
            for i in cart.keys():
                order[i]={"Name":inventory[i]["Name"],"Quantity":cart[i],"Price":inventory[i]["Price"]}
            saveInJson("order.json",order)
        else:
            for i in cart.keys():
                quantity=0
                if(i in order):
                    quantity=int(order[i]["Quantity"])+int(cart[i])
                else:
                    quantity=int(cart[i])
                order[i]={"Name":inventory[i]["Name"],"Quantity":quantity,"Price":inventory[i]["Price"]}
            saveInJson("order.json",order)
    
def payBill(cart):
    viewCart(cart)
    print("Proceed to billing ? Y/N ")
    ch=input()
    if(ch=="Y" or ch=="y"):
        inventory=load("inventory.json")
        if(inventory!=False):
            for i in cart.keys():
                inventory[i]['Quantity']=int(inventory[i]['Quantity'])-cart[i]
            saveInJson("inventory.json",inventory)
            saveInOrders(cart)
            return True
        else:
            print("Cannot proceed with the billing")
    else:
        print("Not proceeded with the payment")
    return False

def price(cart):
    inventory=load("inventory.json")
    if(inventory==False):
        return 0
    s=0
    for i in cart.keys():
        s=s+cart[i]*inventory[i]["Price"]
    return s

def viewCart(cart):
    inventory=load("inventory.json")
    dCart=[["ID","Name","Quantity","Price"]]
    if(inventory!=False):
        for i in cart.keys():
            dItem=[i,inventory[i]["Name"],cart[i],inventory[i]["Price"]]
            dCart.append(dItem)
    print(tabulate(dCart))
    print("The total Price is ",price(cart))

def userMain(): 
    cart={}
    while(True):
        print(tabulate(lUser))
        ch=int(input())
        if(ch==0):
            viewCart(cart)
        elif(ch==1):
            viewInventory()
        elif(ch==2):
            addProductToCart(cart)
        elif(ch==3):
            removeProductFromCart(cart)
        elif(ch==4):
            if(payBill(cart)):
                cart={}
        elif(ch==5):
            print("Thank you for shopping at IMS")
            break
        else:
            print("Invalid Choice. Please re-enter")

def saveInJson(file,dic):
    with open(file,"w") as p:
        json.dump(dic , p)
    p.close() 
    
def addToInventory():
    choice = int(input("Enter 1 if new item is to be added.\nEnter 2 if existing item is to be stocked \n"))

    if choice==1:
        name =  input("Enter item name to be added: ")
        qty = int(input("Enter quantity: "))
        price = int(input("Enter price: "))
        if os.path.isfile('./inventory.json'):
            file = None
            with open("inventory.json","r") as p:
                file=json.load(p)
            p.close()
            
            item_ids = list(map(int,file.keys()))
            new_item_id = max(item_ids) + 1
            file[str(new_item_id)] = {"Name" : name , "Quantity" : qty , "Price" : price}
            saveInJson("inventory.json",file)
        else:
            dic = {}
            item_id = 1001 
            dic[str(item_id)] = {"Name": name,"Quantity" : qty , "Price" : price}
            saveInJson("inventory.json",dic)

    elif choice==2:
        viewInventory()
        item_id = str(input("Please enter the item id of the item to be added: "))
        if os.path.isfile('./inventory.json'):
            try:
                qty = int(input("Enter quantity to be added: "))
                file = None
                with open("inventory.json","r") as p:
                    file=json.load(p)
                p.close()
                if qty>0:
                    file[item_id]["Quantity"]+=qty
                    saveInJson("inventory.json",file)
                else:
                    print("Invalid addition")
            except:
                print("Sorry, no such item")
        else:
            print("Sorry, no such item.")

    else:
        print("Invalid choice. Please re-enter")
        addToInventory()



    


def adminMain():
    while(True):
        print(tabulate(lAdmin))
        choice = int(input("Enter your choice : "))
        
        if choice==1:
            viewInventory()
        elif choice==2:
            addToInventory()
        elif choice==3:
            viewOrders()
        elif choice==4:
            viewStatistics()
        elif choice==5:
            print("Thank you, Admin.")
            break
        else:
            print("Invalid Choice. Please re-enter.")
            
def viewOrders():
    with open("order.json",'r') as file:
        orders=json.load(file)
    file.close()
    keys=list(orders.keys())
    table=[["Id","Name","Quantity","Price"]]
    for i in keys:
        row=[i]
        row.append(orders[i]['Name'])
        row.append(orders[i]['Quantity'])
        row.append(orders[i]['Price'])
        table.append(row)
    print(tabulate(table))
    
def viewStatistics():
    with open("order.json",'r') as file:
        orders=json.load(file)
    file.close()
    
    items = []
    quantities_sold = []
    
    for item in list(orders.keys()):
        items.append(orders[item]["Name"])
        quantities_sold.append(int(orders[item]["Quantity"]))
        
    fig = plt.figure(figsize =(10, 7))
    plt.pie(quantities_sold, labels = items)

    plt.show()
    

    

lUser=[["Press 0 :","To View Cart"],["Press 1 :","View Inventory"],["Press 2 :","Add Product To Cart"],["Press 3 :","Remove Product From Cart"],["Press 4 :","Pay Bill"],["Press 5 :","To Exit"]]
lAdmin=[["Press 1 :","View Inventory"],["Press 2 :","Add To Inventory"],["Press 3 :","View Orders"],["Press 4 :","View Statistics"],["Press 5 :","Exit"]]
    
if __name__ == "__main__":
    main()